# Step 1: Data preprocessing and importing (Getting started)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

we'll start off by the following steps:
- importing necessary libraries
- The data is messy, contain a lot of typos, unnecessary words, punctuations, etc, so we'll clean it using regex

In [ ]:
!pip install rapidfuzz

In [ ]:
import pandas as pd
import numpy as np
#nlp toolkit library helps us in giving us a list of stopwords that we use in cleaning data.
import nltk
from nltk.corpus import stopwords
import re
import random
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import fuzz
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pickle
from sklearn.metrics import classification_report
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

In [ ]:
df = pd.read_excel('/kaggle/input/product-matching-dataset/Product Matching Dataset.xlsx', sheet_name='Dataset')
df = df.sample(frac=1).reset_index(drop=True)
df.head()
# df_master_file = pd.read_excel('/kaggle/input/product-matching-dataset/Product Matching Dataset.xlsx', sheet_name='Master File')
# df.head()

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('arabic')

In [ ]:
stop_words.append('سج')
stop_words.append('سق')
stop_words.append('سعر')
stop_words.append('جديد')
stop_words.append('قديم')
stop_words.append('هام')
stop_words.append('العامريه')
stop_words.append('ادويا')
stop_words.append('ادويه')
stop_words.append('ركز')

In [ ]:
def data_preprocessing(text):
  text = text.replace("أ", "ا")
  text = text.replace("آ", "ا")
  text = text.replace("إ", "ا")
  text = text.replace("ؤ", "و")
  text = text.replace("ئ", "ي")
  text = text.replace("ى", "ي")
  text = text.replace("ة", "ه")

  text = re.sub(r'[^\w\s]', ' ', text)
  text = re.sub(r'(\d*|\s*)جنيه$', ' ', text)
  text = re.sub(r'س\d*ج', " ", text)
  text = re.sub(r'س\d*ق', " ", text)
  text = re.sub(r"(\w)\1+", r'\1', text)
  text = re.sub(r"[ء-ي]*(ركيرل|ركريل)[ء-ي]*", " ", text)
  text = re.sub(r"\d+$", " ", text)
  text = re.sub(r"(\s)\1+", r"\1", text)
  text = text.strip()

  text = ' '.join([word for word in text.split() if word not in stop_words])
  return text



In [ ]:
df['marketplace_product_name_ar'] = df['marketplace_product_name_ar'].apply(data_preprocessing)
df['seller_item_name'] = df['seller_item_name'].apply(data_preprocessing)
df

In [ ]:
df_unique = df.drop_duplicates()
df_unique

In [ ]:
df_unique['label'] = 1
df_unique

we'll put label 1 for the correctly matched drugs, and 0 for the unmatched, the data doesn't contain unmatched drugs, so we'll generate 5 unmatched examples for each drug.
we'll do this so the model can learn correctly the examples that indicate unmatch, and the examples that indicate a match.

In [ ]:
def generate_negative_pairs(master, DF):
    #iterate over dataset df
    #for each sku (medicine), generate negative pairs for it (negative matches)
    arr = defaultdict(list)
    for idx in DF['sku']:
        #generate for example 5 negative matches, being put after the positive pairs,
        #the negative pairs can be (different medicine from master file: different sku, different sku but has high similarity)
        i = 0
        while i < 5:
            choice = random.choice(DF['sku'].unique())
            while choice == idx:
                choice = random.choice(DF['sku'].unique())
            drug_b = random.choice(DF.loc[DF['sku']==choice, 'seller_item_name'].to_list())
            arr[idx].append(drug_b)
            i += 1
    return arr

neg_pairs = generate_negative_pairs(df_master_file, df_unique)
len(neg_pairs)

In [ ]:
new_rows = []

# Iterate over negative_pairs to create new rows
for sku, negatives in neg_pairs.items():
    # Get the correct marketplace product name for this SKU
    correct_name = df.loc[df['sku'] == sku, 'marketplace_product_name_ar'].values[0]

    # Create new rows for each negative pair
    for negative_drug in negatives:
        new_rows.append({'sku': sku, 
                         'marketplace_product_name_ar': correct_name, 
                         'seller_item_name': negative_drug, 
                         'label': 0})

# Convert new_rows into a DataFrame
negative_df = pd.DataFrame(new_rows)
negative_df

In [ ]:
final_df = pd.concat([df_unique, negative_df], ignore_index=True)
final_df

# Step 2: feature extraction and model training
we'll do the following in this step:
- convert the text data into numerical data using tf-idf vectorizer
- create the similarity features that we'll pass to the model as training
- splitting the data and train and evaluate the model

In [ ]:
# TF-IDF vectorizer
tfidf = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
all_names = pd.concat([final_df['marketplace_product_name_ar'], final_df['seller_item_name']])
tfidf.fit(all_names)

def compute_features(row):
    name1, name2 = row['marketplace_product_name_ar'], row['seller_item_name']
    vec1 = tfidf.transform([name1])
    vec2 = tfidf.transform([name2])
    
    cosine_sim = cosine_similarity(vec1, vec2)[0][0]  
    levenshtein = fuzz.ratio(name1, name2) / 100  
    jaro_winkler = fuzz.WRatio(name1, name2) / 100

    return pd.Series([cosine_sim, levenshtein, jaro_winkler])

final_df[['cosine_sim', 'levenshtein', 'jaro_winkler']] = final_df.apply(compute_features, axis=1)
final_df

In [ ]:
X = final_df[['cosine_sim', 'levenshtein', 'jaro_winkler']]
y = final_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

the drugs that has multiple variations (multiple concentrations), we'll consider the output of any of them will be Not sure, so a person could manually check and review them, beceause the model might easily predict a wrong variation of the drug because of how similar and close they are.

In [ ]:
medicine_frequency = defaultdict(int)
for full_drug in df_master_file['product_name']:
    drug = re.split(r'\d+', full_drug)
    medicine_frequency[drug[0]]+=1
    
for full_drug in df_master_file['product_name_ar']:
    drug = re.split(r'\d+', full_drug)
    medicine_frequency[drug[0]]+=1

# medicine_frequency

In [ ]:
def find_best_match(input_name, known_names, model, tfidf):
    similarities = []
    features_list = []
    if 97 <= ord(input_name[0].lower()) <= 122:
        for known_name in known_names['product_name']:
            vec1 = tfidf.transform([input_name])
            vec2 = tfidf.transform([known_name])
    
            cosine_sim = cosine_similarity(vec1, vec2)[0][0]
            levenshtein = fuzz.ratio(input_name, known_name) / 100
            jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
            
            features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
            prob = model.predict_proba(features)[0][1]  # Match probability
            similarities.append((known_name, prob))
            
    else:
        for known_name in known_names['product_name_ar']:
            vec1 = tfidf.transform([input_name])
            vec2 = tfidf.transform([known_name])
            
            cosine_sim = cosine_similarity(vec1, vec2)[0][0]
            levenshtein = fuzz.ratio(input_name, known_name) / 100
            jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
        
            features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
            prob = model.predict_proba(features)[0][1]  # Match probability
            similarities.append((known_name, prob))
    # Get the best match
    best_match = max(similarities, key=lambda x: x[1])
    if (best_match[1]) < 0.95 or medicine_frequency[re.split(r'\d+', best_match[0])[0]] > 1:
        return (best_match, 'Not sure')
    return (best_match, 'sure')

find_best_match('esthl3t MG', df_master_file, xgb_model, tfidf)


the "find_best_match()" function is used for finding the best match for a single medicine.

but what if we want to pass in a sheet of medicines and get the best match for every one of them?
the problem of this is that it'll take time if the sheet is long, so what can we do?
Here comes the power of "parallel programming", we'll use ThreadPools to find the best match for a number of medicines (4 for example) at the same time.

In [ ]:
test_df = pd.read_excel('/kaggle/input/product-matching-dataset/Product Matching Dataset.xlsx', sheet_name='Dataset')
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df.head()

In [ ]:
def find_best_matches(medicines, master, model, tfidf):
    results = []
    for medicine in medicines:
        match = (medicine, find_best_match(medicine, master, model, tfidf))
        print(match)
        results.append(match)
    return results

def find_best_matches_parallel(full_medicines, master, model, tfidf, n_jobs=4):
    chunks = np.array_split(full_medicines, n_jobs)
    
    with ThreadPoolExecutor(max_workers=n_jobs) as executor:
        try:
            results = list(executor.map(lambda chunk: find_best_matches(chunk, master, model, tfidf), chunks))
        except KeyboardInterrupt:
            print("Interrupted! Shutting down threads...")
            executor.shutdown(wait=False, cancel_futures=True)
        return [item for sublist in results for item in sublist]

results = find_best_matches_parallel(lst, df_master_file, xgb_model, tfidf)
results

# Step 3: Model Deployment
- we'll build a class that contains all the steps we did, just for the case of retraining on a new dataset or anything.
- we also saved the class object, which contains the model and other important attributes like: tfidf, medicine_frequency dictionary, which both of them are used in the find_best_match function.

In [ ]:
class MedicineMatcher():
    def __init__(self, model, tfidf, data_for_training, master_file, medicine_frequency):
        self.model = model
        self.tfidf = tfidf
        self.data = data_for_training
        self.master = master_file
        self.m_freq = medicine_frequency

    def __generate_stop_words(self):
        nltk.download('stopwords')
        stop_words = stopwords.words('arabic')
        stop_words.append('سج')
        stop_words.append('سق')
        stop_words.append('سعر')
        stop_words.append('جديد')
        stop_words.append('قديم')
        stop_words.append('هام')
        stop_words.append('العامريه')
        stop_words.append('ادويا')
        stop_words.append('ادويه')
        stop_words.append('ركز')
        return stop_words

    def __data_cleaning(self, text):
        text = text.replace("أ", "ا")
        text = text.replace("آ", "ا")
        text = text.replace("إ", "ا")
        text = text.replace("ؤ", "و")
        text = text.replace("ئ", "ي")
        text = text.replace("ى", "ي")
        text = text.replace("ة", "ه")
        
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'(\d*|\s*)جنيه$', ' ', text)
        text = re.sub(r'س\d*ج', " ", text)
        text = re.sub(r'س\d*ق', " ", text)
        text = re.sub(r"(\w)\1+", r'\1', text)
        text = re.sub(r"[ء-ي]*(ركيرل|ركريل)[ء-ي]*", " ", text)
        text = re.sub(r"\d+$", " ", text)
        text = re.sub(r"(\s)\1+", r"\1", text)
        text = text.strip()

        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text


    def __generate_negative_pairs(self, DF):
        arr = defaultdict(list)
        for idx in DF['sku']:
            #generate for example 5 negative matches, being put after the positive pairs,
            #the negative pairs can be (different medicine from master file: different sku, different sku but has high similarity)
            i = 0
            while i < 5:
                choice = random.choice(DF['sku'].unique())
                while choice == idx:
                    choice = random.choice(DF['sku'].unique())
                drug_b = random.choice(DF.loc[DF['sku']==choice, 'seller_item_name'].to_list())
                arr[idx].append(drug_b)
                i += 1
                
        new_rows = []

        # Iterate over negative_pairs to create new rows
        for sku, negatives in arr.items():
            # Get the correct marketplace product name for this SKU
            correct_name = self.data.loc[self.data['sku'] == sku, 'marketplace_product_name_ar'].values[0]
        
            # Create new rows for each negative pair
            for negative_drug in negatives:
                new_rows.append({'sku': sku, 
                                 'marketplace_product_name_ar': correct_name, 
                                 'seller_item_name': negative_drug, 
                                 'label': 0})
        
        # Convert new_rows into a DataFrame
        negative_df = pd.DataFrame(new_rows)
        return negative_df

    def data_preprocessing(self):
        stop_words = self.__generate_stop_words()
        self.data['marketplace_product_name_ar'] = self.data['marketplace_product_name_ar'].apply(self.__data_cleaning)
        self.data['seller_item_name'] = self.data['seller_item_name'].apply(self.__data_cleaning)
        df_unique = self.data.drop_duplicates()
        df_unique['label'] = 1
        negative_df = self.__generate_negative_pairs(df_unique)
        self.data = pd.concat([df_unique, negative_df], ignore_index=True)



    def __compute_similarities(self, row):
        name1, name2 = row['marketplace_product_name_ar'], row['seller_item_name']
        vec1 = self.tfidf.transform([name1])
        vec2 = self.tfidf.transform([name2])

        cosine_sim = cosine_similarity(vec1, vec2)[0][0]
        levenshtein = fuzz.ratio(name1, name2) / 100
        jaro_winkler = fuzz.WRatio(name1, name2) / 100

        return pd.Series([cosine_sim, levenshtein, jaro_winkler])
        
    def __medicineFrequency(self):
        for full_drug in self.master['product_name']:
            drug = re.split(r'\d+', full_drug)
            self.m_freq[drug[0]] += 1

        for full_drug in self.master['product_name_ar']:
            drug = re.split(r'\d+', full_drug)
            self.m_freq[drug[0]] += 1

    def Fit(self):
        # data preparation step
        self.tfidf = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
        all_names = pd.concat([self.data['marketplace_product_name_ar'], self.data['seller_item_name']])
        self.tfidf.fit(all_names)
        self.data[['cosine_sim', 'levenshtein', 'jaro_winkler']] = self.data.apply(self.__compute_similarities, axis=1)
        self.__medicineFrequency()
        
        #training
        X = self.data[['cosine_sim', 'levenshtein', 'jaro_winkler']]
        y = self.data['label']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        self.model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        self.model.fit(X_train, y_train)

    def evaluate(self):
        y_pred = xgb_model.predict(X_test)
        print(classification_report(y_test, y_pred))

    def find_best_match(self, input_name):
        """
        input_name: medicine name to be matched
        """
        similarities = []
        features_list = []
        if 97 <= ord(input_name[0].lower()) <= 122:
            for known_name in self.master['product_name']:
                vec1 = self.tfidf.transform([input_name])
                vec2 = self.tfidf.transform([known_name])
        
                cosine_sim = cosine_similarity(vec1, vec2)[0][0]
                levenshtein = fuzz.ratio(input_name, known_name) / 100
                jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
                
                features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
                prob = self.model.predict_proba(features)[0][1]  # Match probability
                similarities.append((known_name, prob))
                
        else:
            for known_name in self.master['product_name_ar']:
                vec1 = self.tfidf.transform([input_name])
                vec2 = self.tfidf.transform([known_name])
                
                cosine_sim = cosine_similarity(vec1, vec2)[0][0]
                levenshtein = fuzz.ratio(input_name, known_name) / 100
                jaro_winkler = fuzz.WRatio(input_name, known_name) / 100
            
                features = np.array([[cosine_sim, levenshtein, jaro_winkler]])
                prob = self.model.predict_proba(features)[0][1]  # Match probability
                similarities.append((known_name, prob))
        # Get the best match
        best_match = max(similarities, key=lambda x: x[1])
        if (best_match[1]) < 0.95 or self.m_freq[re.split(r'\d+', best_match[0])[0]] > 1:
            return (best_match, 'Not sure')
        return (best_match, 'sure')

    
    def __find_best_matches(self, medicines):
        results = []
        for medicine in medicines:
            match = (medicine, self.find_best_match(medicine))
            print(match)
            results.append(match)
        return results
    
    def find_best_matches_parallel(self, full_medicines, n_jobs=4):
        """
        full_medicines: a list of medicine names that need to be matched
        n_jobs: number of jobs to be done in parallel using ThreadPooling, used in splitting
        the list into chunks so each core will handle a chunk
        """
        chunks = np.array_split(full_medicines, n_jobs)
        
        with ThreadPoolExecutor(max_workers=n_jobs) as executor:
            try:
                results = list(executor.map(lambda chunk: self.__find_best_matches(chunk), chunks))
            except KeyboardInterrupt:
                print("Interrupted! Shutting down threads...")
                executor.shutdown(wait=False, cancel_futures=True)
            return [item for sublist in results for item in sublist]


In [ ]:
medicine_matcher = MedicineMatcher(xgb_model, tfidf, final_df, df_master_file, medicine_frequency)

In [ ]:
medicine_matcher.find_best_match('brofen')

In [ ]:
# save the current object
with open("medicine_matcher.pkl", "wb") as f:
    pickle.dump(medicine_matcher, f)

In [ ]:
# Load the saved class
with open("/kaggle/input/medicinematcher/medicine_matcher.pkl", "rb") as f:
    matcher = pickle.load(f)

In [ ]:
matcher.find_best_match('estoh3lt')